In [31]:
#Simulador do potencial devido a uma carga pontual em espaço vazio e entre placas paralelas
#Autor: Danilo Lessa Bernardineli

#Este programa faz a simulação do potencial eletrostático pela eq. de Poisson
#bem como do campo elétrico através do método de relaxação de Jacobi em duas situações diferentes:
# - Carga pontual em um espaço vazio
# - Carga pontual entre duas placas paralelas com tensão constante

#Dependências
import matplotlib.pyplot as plt
import numpy as np

#Instruções de uso: 
#1 - Coloque True ou False no usarContorno
#2(opcional) - Altere as condições iniciais e parâmetro a gosto
#3 - Dê shift-enter até o final e espere sair o resultado

#%matplotlib inline #descomente caso queira gráficos no proprio IPython ao invés de janelas

In [54]:
#Bloco de condições iniciais

#Coloque True caso queira simular a carga pontual com a presença de placas paralelas
#Coloque False caso queira simular a carga pontual em um espaço vazio
usarContorno = False

#Tensão nas duas placas
Vp = 1
Vn = -1

#Carga pontual
q = 2

In [55]:
#Bloco de parâmetros de simulação

#Número de iterações pelo relaxamento de Jacobi
Ns = 300

#Tamanho da matriz de simulação: N=10*m. Altere somente o m pois ele é utilizado
#para manter a proporção de distância entre as placas bem como a carga
m = 5
N = 10 * m
n = N-1

#localização da carga
ind_q = (5*m, 1*m)

#Eixos x e y
x = np.arange(0, N)
y = np.arange(0, N)

#Condições de contorno
Vc = np.zeros((N, N))
Vc_inds = []
#Condições de contorno no caso das placas
if usarContorno:
    for i in range(2*m, N-2*m):
        Vc[3*m, i] = Vp
        Vc[7*m, i] = Vn
        Vc_inds.append((3*m, i))
        Vc_inds.append((7*m, i))

In [56]:
#cálculo do potencial eletrostático através do método de relaxamento de Jacobi
#Inicialização da matriz de resultados
V = np.zeros((N, N, Ns))
V[:,:,0] = Vc

#Lista contendo a convergência para cada iteração
conv_l = np.zeros((Ns, 1))

for n in range(1, Ns):
    conv = 0
    
    for i in range(1, N-1):
        for j in range(1, N-1):
            
            if  (i,j) in Vc_inds: #Caso caia na condição de contorno
                V[i, j, n] = Vc[i, j]    
                
            elif (i, j) == ind_q: #Caso caia na localização da carga pontual
                V[i, j, n] = (V[i+1,j, n-1]+V[i-1,j, n-1]+V[i,j+1, n-1]+V[i,j-1, n-1] + q)/4 
                
            else: #Espaço livre
                V[i, j, n] = (V[i+1,j,n-1] + V[i-1,j,n-1] + V[i,j+1, n-1] + V[i,j-1,n-1])/4
                
            #Calcular o termo de convergência
            conv += np.abs(V[i, j, n] - V[i, j, n-1])
            
    conv_l[n] = conv

In [57]:
#Obtenção do campo elétrico
Ex = np.zeros((N, N))
Ey = np.zeros((N, N))

n = N-1
for i in range(1, N-1):
    #Campo elétrico nos limites da matriz de simulação
    Ex[0, i] = -(V[1, i,Ns-1] - V[0, i,Ns-1]) / 2
    Ex[n, i] = -(V[n, i,Ns-1] - V[n-1, i,Ns-1]) / 2
    Ey[i, 0] = (V[i, 1,Ns-1] - V[i, 0,Ns-1]) / 2
    Ey[i, n] = (V[i, n,Ns-1] - V[i, n-1,Ns-1]) / 2
    
    for j in range(1, n):
        if (i,j) in Vc_inds: #Não há campo elétrico em um condutor
            continue
        #Campo elétrico no resto do espaço
        Ex[i,j] = -(V[i-1,j,Ns-1]-V[i+1,j,Ns-1]) / 2
        Ey[i,j] = (V[i,j-1,Ns-1]-V[i,j+1,Ns-1]) / 2

In [58]:
#Graficar potencial eletrostático e campo elétrico
plt.matshow(V[:,:,Ns-1])
cb = plt.colorbar()
cb.set_label("Potencial eletrostático")
plt.quiver(y,x,Ey,Ex, units='xy')
plt.title("Potencial eletrostático e campo elétrico")
plt.show()

In [60]:
#Graficar convergência por iteração
plt.plot(np.arange(0, Ns), conv_l, '+-')
plt.xlabel("Número de iterações")
plt.ylabel("Convergência")
plt.title("Convergência em cada termo")
plt.show()

In [61]:
0.2 / (50 * 50)

8e-05